# Amazon SageMaker Autopilot Candidate Definition Notebook

This notebook was automatically generated by the AutoML job **mangrove-2013**.
This notebook allows you to customize the candidate definitions and execute the SageMaker Autopilot workflow.

The dataset has **8** columns and the column named **label** is used as
the target column. This is being treated as a **BinaryClassification** problem. The dataset also has **2** classes.
This notebook will build a **[BinaryClassification](https://en.wikipedia.org/wiki/Binary_classification)** model that
**maximizes** the "**AUC**" quality metric of the trained models.
The "**AUC**" metric applies to binary classification with a positive and negative class. It calculates the area under the ROC curve which is created by plotting the true positive rate against the false positive rate..

As part of the AutoML job, the input dataset has been randomly split into two pieces, one for **training** and one for
**validation**. This notebook helps you inspect and modify the data transformation approaches proposed by Amazon SageMaker Autopilot. You can interactively
train the data transformation models and use them to transform the data. Finally, you can execute a multiple algorithm hyperparameter optimization (multi-algo HPO)
job that helps you find the best model for your dataset by jointly optimizing the data transformations and machine learning algorithms.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>
Look for sections like this for recommended settings that you can change.
</div>


---

## Contents

1. [Sagemaker Setup](#Sagemaker-Setup)
    1. [Downloading Generated Candidates](#Downloading-Generated-Modules)
    1. [SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration](#SageMaker-Autopilot-Job-and-Amazon-Simple-Storage-Service-(Amazon-S3)-Configuration)
1. [Candidate Pipelines](#Candidate-Pipelines)
    1. [Generated Candidates](#Generated-Candidates)
    1. [Selected Candidates](#Selected-Candidates)
1. [Executing the Candidate Pipelines](#Executing-the-Candidate-Pipelines)
    1. [Run Data Transformation Steps](#Run-Data-Transformation-Steps)
    1. [Multi Algorithm Hyperparameter Tuning](#Multi-Algorithm-Hyperparameter-Tuning)
1. [Model Selection and Deployment](#Model-Selection-and-Deployment)
    1. [Tuning Job Result Overview](#Tuning-Job-Result-Overview)
    1. [Model Deployment](#Model-Deployment)

---

## Sagemaker Setup

Before you launch the SageMaker Autopilot jobs, we'll setup the environment for Amazon SageMaker
- Check environment & dependencies.
- Create a few helper objects/function to organize input/output data and SageMaker sessions.

**Minimal Environment Requirements**

- Jupyter: Tested on `JupyterLab 1.0.6`, `jupyter_core 4.5.0` and `IPython 6.4.0`
- Kernel: `conda_python3`
- Dependencies required
  - `sagemaker-python-sdk>=2.40.0`
    - Use `!pip install sagemaker==2.40.0` to download this dependency.
    - Kernel may need to be restarted after download.
- Expected Execution Role/permission
  - S3 access to the bucket that stores the notebook.

### Downloading Generated Modules
Download the generated data transformation modules and an SageMaker Autopilot helper module used by this notebook.
Those artifacts will be downloaded to **mangrove-2013-artifacts** folder.

In [2]:
!mkdir -p mangrove-2013-artifacts
!aws s3 sync s3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013/sagemaker-automl-candidates/mangrove-2013-pr-1-3fbd4f34dd314d0dbd0bf3ad17f37ce4f4b731e72ed2/generated_module mangrove-2013-artifacts/generated_module --only-show-errors
!aws s3 sync s3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013/sagemaker-automl-candidates/mangrove-2013-pr-1-3fbd4f34dd314d0dbd0bf3ad17f37ce4f4b731e72ed2/notebooks/sagemaker_automl mangrove-2013-artifacts/sagemaker_automl --only-show-errors

import sys
sys.path.append("mangrove-2013-artifacts")

### SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration

The following configuration has been derived from the SageMaker Autopilot job. These items configure where this notebook will
look for generated candidates, and where input and output data is stored on Amazon S3.

In [3]:
from sagemaker_automl import uid, AutoMLLocalRunConfig

# Where the preprocessed data from the existing AutoML job is stored
BASE_AUTOML_JOB_NAME = 'mangrove-2013'
BASE_AUTOML_JOB_CONFIG = {
    'automl_job_name': BASE_AUTOML_JOB_NAME,
    'automl_output_s3_base_path': 's3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013',
    'data_transformer_image_repo_version': '2.4-1-cpu-py3',
    'algo_image_repo_versions': {'xgboost': '1.2-2-cpu-py3', 'linear-learner': 'latest', 'mlp': 'training-cpu'},
    'algo_inference_image_repo_versions': {'xgboost': '1.2-2-cpu-py3', 'linear-learner': 'latest', 'mlp': 'inference-cpu'}
}

# Path conventions of the output data storage path from the local AutoML job run of this notebook
LOCAL_AUTOML_JOB_NAME = 'mangrove-2-notebook-run-{}'.format(uid())
LOCAL_AUTOML_JOB_CONFIG = {
    'local_automl_job_name': LOCAL_AUTOML_JOB_NAME,
    'local_automl_job_output_s3_base_path': 's3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013/{}'.format(LOCAL_AUTOML_JOB_NAME),
    'data_processing_model_dir': 'data-processor-models',
    'data_processing_transformed_output_dir': 'transformed-data',
    'multi_algo_tuning_output_dir': 'multi-algo-tuning'
}

AUTOML_LOCAL_RUN_CONFIG = AutoMLLocalRunConfig(
    role='arn:aws:iam::766126314930:role/sm-service-access',
    base_automl_job_config=BASE_AUTOML_JOB_CONFIG,
    local_automl_job_config=LOCAL_AUTOML_JOB_CONFIG,
    security_config={'EnableInterContainerTrafficEncryption': False, 'VpcConfig': {}})

AUTOML_LOCAL_RUN_CONFIG.display()

This notebook is initialized to use the following configuration: 
        <table>
        <tr><th colspan=2>Name</th><th>Value</th></tr>
        <tr><th>General</th><th>Role</th><td>arn:aws:iam::766126314930:role/sm-service-access</td></tr>
        <tr><th rowspan=2>Base AutoML Job</th><th>Job Name</th><td>mangrove-2013</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013</td></tr>
        <tr><th rowspan=5>Interactive Job</th><th>Job Name</th><td>mangrove-2-notebook-run-21-16-54-33</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013/mangrove-2-notebook-run-21-16-54-33</td></tr>
        <tr><th>Data Processing Trained Model Directory</th><td>s3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013/mangrove-2-notebook-run-21-16-54-33/data-processor-models</td></tr>
        <tr><th>Data Processing Transformed Output</th><td>s3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013/mangrove-2-notebook-run-21-16-54-33/transformed-data</td></tr>
        <tr><th>Algo Tuning Model Output Directory</th><td>s3://sagemaker-gis/LANDSAT_LC08_C01_T1_SR/Year2013/mangrove-2013/mangrove-2-notebook-run-21-16-54-33/multi-algo-tuning</td></tr>
        </table>
        

## Candidate Pipelines

The `AutoMLLocalRunner` keeps track of selected candidates and automates many of the steps needed to execute feature engineering and tuning steps.

In [4]:
from sagemaker_automl import AutoMLInteractiveRunner, AutoMLLocalCandidate

automl_interactive_runner = AutoMLInteractiveRunner(AUTOML_LOCAL_RUN_CONFIG)

### Generated Candidates

The SageMaker Autopilot Job has analyzed the dataset and has generated **7** machine learning
pipeline(s) that use **3** algorithm(s). Each pipeline contains a set of feature transformers and an
algorithm.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. The resource configuration: instance type & count
1. Select candidate pipeline definitions by cells
1. The linked data transformation script can be reviewed and updated. Please refer to the [README.md](./mangrove-2013-artifacts/generated_module/README.md) for detailed customization instructions.
</div>

**[dpp0-xgboost](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp0.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [5]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp0",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-10-21 16:54:46,582 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:46,604 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:46,606 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:46,674 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:46,675 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:46,693 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp1-xgboost](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp1.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [6]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp1",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-10-21 16:54:48,014 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:48,044 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:48,046 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:48,061 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:48,063 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:48,082 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp2-linear-learner](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp2.py)**: This data transformation strategy first transforms 'numeric' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [7]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp2",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-10-21 16:54:49,463 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:49,479 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:49,480 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:49,482 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.
2021-10-21 16:54:49,496 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:49,497 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:49,498 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.
2021-10-21 16:54:49,515 INFO sagemaker.image_uris: Ignoring unnecessary instance t

**[dpp3-xgboost](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp3.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [8]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp3",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-10-21 16:54:50,991 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:51,011 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:51,012 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:51,023 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:51,024 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:51,035 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp4-xgboost](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp4.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [9]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp4",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-10-21 16:54:52,984 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:53,000 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:53,003 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:53,018 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:53,020 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:53,035 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp5-xgboost](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp5.py)**: This data transformation strategy first transforms 'numeric' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [10]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp5",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2021-10-21 16:54:54,315 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:54,330 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:54,331 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:54,343 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:54,344 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:54,425 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.


**[dpp6-mlp](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp6.py)**: This data transformation strategy transforms 'numeric' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *mlp* model. Here is the definition:

In [11]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp6",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": True,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "mlp",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "candidate_specific_static_hyperparameters": {
            "num_categorical_features": '0',
        }
    }
})

2021-10-21 16:54:55,831 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:55,849 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:55,850 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:55,852 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: training-cpu.
2021-10-21 16:54:55,862 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 16:54:55,864 INFO sagemaker.image_uris: Same images used for training and inference. Defaulting to image scope: inference.
2021-10-21 16:54:55,865 WARNING sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: inference-cpu.
2021-10-21 16:54:55,877 INFO sagemaker.image_uris: Ignoring unnecessa

### Selected Candidates

You have selected the following candidates (please run the cell below and click on the feature transformer links for details):

In [13]:
automl_interactive_runner.display_candidates()

Candidate Name,Algorithm,Feature Transformer
dpp0-xgboost,xgboost,dpp0.py
dpp1-xgboost,xgboost,dpp1.py
dpp2-linear-learner,linear-learner,dpp2.py
dpp3-xgboost,xgboost,dpp3.py
dpp4-xgboost,xgboost,dpp4.py
dpp5-xgboost,xgboost,dpp5.py
dpp6-mlp,mlp,dpp6.py


The feature engineering pipeline consists of two SageMaker jobs:

1. Generated trainable data transformer Python modules like [dpp0.py](mangrove-2013-artifacts/generated_module/candidate_data_processors/dpp0.py), which has been downloaded to the local file system
2. A **training** job to train the data transformers
3. A **batch transform** job to apply the trained transformation to the dataset to generate the algorithm compatible data

The transformers and its training pipeline are built using open sourced **[sagemaker-scikit-learn-container][]** and **[sagemaker-scikit-learn-extension][]**.

[sagemaker-scikit-learn-container]: https://github.com/aws/sagemaker-scikit-learn-container
[sagemaker-scikit-learn-extension]: https://github.com/aws/sagemaker-scikit-learn-extension

## Executing the Candidate Pipelines

Each candidate pipeline consists of two steps, feature transformation and algorithm training.
For efficiency first execute the feature transformation step which will generate a featurized dataset on S3
for each pipeline.

After each featurized dataset is prepared, execute a multi-algorithm tuning job that will run tuning jobs
in parallel for each pipeline. This tuning job will execute training jobs to find the best set of
hyper-parameters for each pipeline, as well as finding the overall best performing pipeline.

### Run Data Transformation Steps

Now you are ready to start execution all data transformation steps.  The cell below may take some time to finish,
feel free to go grab a cup of coffee. To expedite the process you can set the number of `parallel_jobs` to be up to 10.
Please check the account limits to increase the limits before increasing the number of jobs to run in parallel.

In [16]:
%%time
automl_interactive_runner.fit_data_transformers(parallel_jobs=7)

2021-10-21 17:28:33,715 INFO root: [Worker_3:dpp3-xgboost]Executing step: train_data_transformer
2021-10-21 17:28:33,717 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-10-21 17:28:33,722 INFO root: [Worker_4:dpp4-xgboost]Executing step: train_data_transformer
2021-10-21 17:28:33,727 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-10-21 17:28:33,870 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 17:28:33,871 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 17:28:34,005 INFO sagemaker: Creating training-job with name: mangrove-2-notebook-run-21-16-54-33-dpp4-train-21-17-28-31
2021-10-21 17:28:34,009 INFO sagemaker: Creating training-job with name: mangrove-2-notebook-run-21-16-54-33-dpp3-train-21-17-28-31

2021-10-21 17:28:34 Starting - Starting the training job
2021-10-21 17:28:34 Starting - Starting the training job

### Multi Algorithm Hyperparameter Tuning

Now that the algorithm compatible transformed datasets are ready, you can start the multi-algorithm model tuning job
to find the best predictive model. The following algorithm training job configuration for each
algorithm is auto-generated by the AutoML Job as part of the recommendation.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Hyperparameter ranges
2. Objective metrics
3. Recommended static algorithm hyperparameters.

Please refers to [Xgboost tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) and [Linear learner tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner-tuning.html) for detailed explanations of the parameters.
</div>

The AutoML recommendation job has recommended the following hyperparameters, objectives and accuracy metrics for
the algorithm and problem type:

In [17]:
ALGORITHM_OBJECTIVE_METRICS = {
    'xgboost': 'validation:auc',
    'linear-learner': 'validation:roc_auc_score',
    'mlp': 'validation:roc_auc',
}

STATIC_HYPERPARAMETERS = {
    'xgboost': {
        'objective': 'binary:logistic',
        'eval_metric': 'accuracy,f1_binary,auc',
        '_kfold': 5,
        '_num_cv_round': 3,
    },
    'linear-learner': {
        'predictor_type': 'binary_classifier',
        'loss': 'logistic',
        'mini_batch_size': 800,
        'binary_classifier_model_selection_criteria': 'loss_function',
        'num_models': 1,
    },
    'mlp': {
        'problem_type': 'binary_classification',
        'positive_example_weight_mult': 'auto',
        'ml_application': 'mlp',
        'use_batchnorm': 'true',
        'activation': 'relu',
        'warmup_epochs': 10,
        'reporting_metrics': 'accuracy,binary_f_1,roc_auc',
        'eval_metric': 'roc_auc',
        'kfold': 5,
        'num_cv_rounds': 3,
    },
}

The following tunable hyperparameters search ranges are recommended for the Multi-Algo tuning job:

In [18]:
from sagemaker.parameter import CategoricalParameter, ContinuousParameter, IntegerParameter

ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES = {
    'xgboost': {
        'num_round': IntegerParameter(64, 1024, scaling_type='Logarithmic'),
        'max_depth': IntegerParameter(2, 8, scaling_type='Logarithmic'),
        'eta': ContinuousParameter(1e-3, 1.0, scaling_type='Logarithmic'),
        'gamma': ContinuousParameter(1e-6, 64.0, scaling_type='Logarithmic'),
        'min_child_weight': ContinuousParameter(1e-6, 32.0, scaling_type='Logarithmic'),
        'subsample': ContinuousParameter(0.5, 1.0, scaling_type='Linear'),
        'colsample_bytree': ContinuousParameter(0.3, 1.0, scaling_type='Linear'),
        'lambda': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
        'alpha': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
    },
    'linear-learner': {
        'wd': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'l1': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'learning_rate': ContinuousParameter(1e-5, 1.0, scaling_type='Logarithmic'),
    },
    'mlp': {
        'mini_batch_size': IntegerParameter(128, 512, scaling_type='Linear'),
        'learning_rate': ContinuousParameter(1e-6, 1e-2, scaling_type='Logarithmic'),
        'weight_decay': ContinuousParameter(1e-12, 1e-2, scaling_type='Logarithmic'),
        'dropout_prob': ContinuousParameter(0.25, 0.5, scaling_type='Linear'),
        'embedding_size_factor': ContinuousParameter(0.65, 0.95, scaling_type='Linear'),
        'network_type': CategoricalParameter(['feedforward', 'widedeep']),
        'layers': CategoricalParameter(['256', '50, 25', '100, 50', '200, 100', '256, 128', '300, 150', '200, 100, 50']),
    },
}

#### Prepare Multi-Algorithm Tuner Input

To use the multi-algorithm HPO tuner, prepare some inputs and parameters. Prepare a dictionary whose key is the name of the trained pipeline candidates and the values are respectively:

1. Estimators for the recommended algorithm
2. Hyperparameters search ranges
3. Objective metrics

In [19]:
multi_algo_tuning_parameters = automl_interactive_runner.prepare_multi_algo_parameters(
    objective_metrics=ALGORITHM_OBJECTIVE_METRICS,
    static_hyperparameters=STATIC_HYPERPARAMETERS,
    hyperparameters_search_ranges=ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES)

Below you prepare the inputs data to the multi-algo tuner:

In [20]:
multi_algo_tuning_inputs = automl_interactive_runner.prepare_multi_algo_inputs()

#### Create Multi-Algorithm Tuner

With the recommended Hyperparameter ranges and the transformed dataset, create a multi-algorithm model tuning job
that coordinates hyper parameter optimizations across the different possible algorithms and feature processing strategies.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Tuner strategy: [Bayesian](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization), [Random Search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)
2. Objective type: `Minimize`, `Maximize`, see [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization)
3. Max Job size: the max number of training jobs HPO would be launching to run experiments. Note the default value is **250**
    which is the default of the managed flow.
4. Parallelism. Number of jobs that will be executed in parallel. Higher value will expedite the tuning process.
    Please check the account limits to increase the limits before increasing the number of jobs to run in parallel
5. Please use a different tuning job name if you re-run this cell after applied customizations.
</div>

In [23]:
from sagemaker.tuner import HyperparameterTuner

base_tuning_job_name = "{}-tuning".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name)

tuner = HyperparameterTuner.create(
    base_tuning_job_name=base_tuning_job_name,
    strategy='Bayesian',
    objective_type='Maximize',
    max_parallel_jobs=10,
    max_jobs=50,
    **multi_algo_tuning_parameters,
)

#### Run Multi-Algorithm Tuning

Now you are ready to start running the **Multi-Algo Tuning** job. After the job is finished, store the tuning job name which you use to select models in the next section.
The tuning process will take some time, please track the progress in the Amazon SageMaker Hyperparameter tuning jobs console.

In [ ]:
%%time
from IPython.display import display, Markdown

# Run tuning
tuner.fit(inputs=multi_algo_tuning_inputs, include_cls_metadata=None)
tuning_job_name = tuner.latest_tuning_job.name

display(
    Markdown(f"Tuning Job {tuning_job_name} started, please track the progress from [here](https://{AUTOML_LOCAL_RUN_CONFIG.region}.console.aws.amazon.com/sagemaker/home?region={AUTOML_LOCAL_RUN_CONFIG.region}#/hyper-tuning-jobs/{tuning_job_name})"))

# Wait for tuning job to finish
tuner.wait()

2021-10-21 20:16:09,756 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-10-21 20:16:09,803 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 20:16:09,804 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-10-21 20:16:09,819 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 20:16:09,820 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-10-21 20:16:09,834 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 20:16:09,836 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-10-21 20:16:09,846 INFO sagemaker.image_uris: Ignoring unnecessary instance type: None.
2021-10-21 20:16:09,847 INFO sagemaker.image_uris: Defaulting to the only supported framework/algorithm version: latest.
2021-10-21 20:16:09,864

Tuning Job mangrove-2-notebook--211021-2016 started, please track the progress from [here](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/hyper-tuning-jobs/mangrove-2-notebook--211021-2016)

!
CPU times: user 6 s, sys: 425 ms, total: 6.43 s
Wall time: 1h 17min 39s


## Model Selection and Deployment

This section guides you through the model selection process. Afterward, you construct an inference pipeline
on Amazon SageMaker to host the best candidate.

Because you executed the feature transformation and algorithm training in two separate steps, you now need to manually
link each trained model with the feature transformer that it is associated with. When running a regular Amazon
SageMaker Autopilot job, this will automatically be done for you.

### Tuning Job Result Overview

The performance of each candidate pipeline can be viewed as a Pandas dataframe. For more interactive usage please
refers to [model tuning monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html).

In [36]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

SAGEMAKER_SESSION = AUTOML_LOCAL_RUN_CONFIG.sagemaker_session
SAGEMAKER_ROLE = AUTOML_LOCAL_RUN_CONFIG.role

tuner_analytics = HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# select the columns to display and rename
select_columns = ["TrainingJobDefinitionName", "FinalObjectiveValue", "TrainingElapsedTimeSeconds"]
rename_columns = {
    "TrainingJobDefinitionName": "candidate",
    "FinalObjectiveValue": "AUC", 
    "TrainingElapsedTimeSeconds": "run_time"  
}

# Show detailed analytics for the top 5 models
df_tuning_job_analytics.rename(columns=rename_columns)[rename_columns.values()].set_index("candidate").head(5)

,AUC,run_time
candidate,,
dpp6-mlp,0.96008,2711.0
dpp4-xgboost,0.95236,385.0
dpp3-xgboost,0.95095,202.0
dpp4-xgboost,0.95069,458.0
dpp3-xgboost,0.95015,361.0


The best training job can be selected as below:

<div class="alert alert-info"> 💡 <strong>Tips: </strong>
You could select alternative job by using the value from `TrainingJobName` column above and assign to `best_training_job` below
</div>

In [37]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)
best_training_job = attached_tuner.best_training_job()

print("Best Multi Algorithm HPO training job name is {}".format(best_training_job))

Best Multi Algorithm HPO training job name is mangrove-2-notebook--211021-2016-012-500271c8


In [39]:
# Number of models built per algorithm
df_tuning_job_analytics.TrainingJobDefinitionName.value_counts()

dpp4-xgboost           14
dpp5-xgboost            8
dpp3-xgboost            6
dpp0-xgboost            6
dpp1-xgboost            6
dpp2-linear-learner     5
dpp6-mlp                5
Name: TrainingJobDefinitionName, dtype: int64

In [40]:
df_tuning_job_analytics.loc[df_tuning_job_analytics.TrainingJobName==best_training_job].T.dropna()

,38
TrainingJobName,mangrove-2-notebook--211021-2016-012-500271c8
TrainingJobStatus,Completed
FinalObjectiveValue,0.96008
TrainingStartTime,2021-10-21 20:22:55+00:00
TrainingEndTime,2021-10-21 21:08:06+00:00
TrainingElapsedTimeSeconds,2711
TrainingJobDefinitionName,dpp6-mlp
dropout_prob,0.415778
embedding_size_factor,0.849226
layers,256


### Linking Best Training Job with Feature Pipelines

Finally, deploy the best training job to Amazon SageMaker along with its companion feature engineering models.
At the end of the section, you get an endpoint that's ready to serve online inference or start batch transform jobs!

Deploy a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/pipeline.html) that has multiple containers of the following:

1. Data Transformation Container: a container built from the model we selected and trained during the data transformer sections
2. Algorithm Container: a container built from the trained model we selected above from the best HPO training job.
3. Inverse Label Transformer Container: a container that converts numerical intermediate prediction value back to non-numerical label value.

Get both best data transformation model and algorithm model from best training job and create an pipeline model:

In [45]:
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel
from sagemaker_automl import select_inference_output

# Get a data transformation model from chosen candidate
best_candidate = automl_interactive_runner.choose_candidate(df_tuning_job_analytics, best_training_job)
best_data_transformer_model = best_candidate.get_data_transformer_model(role=SAGEMAKER_ROLE, sagemaker_session=SAGEMAKER_SESSION)

# Our first data transformation container will always return recordio-protobuf format
best_data_transformer_model.env["SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT"] = 'application/x-recordio-protobuf'
# Add environment variable for sparse encoding
if best_candidate.data_transformer_step.sparse_encoding:
    best_data_transformer_model.env["AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF"] = '1'

# Get a algo model from chosen training job of the candidate
algo_estimator = Estimator.attach(best_training_job)
best_algo_model = algo_estimator.create_model(**best_candidate.algo_step.get_inference_container_config())

# Final pipeline model is composed of data transformation models and algo model and an
# inverse label transform model if we need to transform the intermediates back to non-numerical value
model_containers = [best_data_transformer_model, best_algo_model]
if best_candidate.transforms_label:
    model_containers.append(best_candidate.get_data_transformer_model(
        transform_mode="inverse-label-transform",
        role=SAGEMAKER_ROLE,
        sagemaker_session=SAGEMAKER_SESSION))

# This model can emit response ['predicted_label', 'probability', 'labels', 'probabilities']. To enable the model to emit one or more
# of the response content, pass the keys to `output_key` keyword argument in the select_inference_output method.

model_containers = select_inference_output("BinaryClassification", model_containers, output_keys=['predicted_label'])


pipeline_model = PipelineModel(
    name=f"mangrove-automl-2013",
    role=SAGEMAKER_ROLE,
    models=model_containers,
    vpc_config=AUTOML_LOCAL_RUN_CONFIG.vpc_config)

2021-10-22 01:57:14,565 INFO root: Chosen Data Processing pipeline candidate name is dpp6-mlp

2021-10-21 17:32:49 Starting - Preparing the instances for training
2021-10-21 17:32:49 Downloading - Downloading input data
2021-10-21 17:32:49 Training - Training image download completed. Training in progress.
2021-10-21 17:32:49 Uploading - Uploading generated training model
2021-10-21 17:32:49 Completed - Training job completed

2021-10-21 21:08:06 Starting - Preparing the instances for training
2021-10-21 21:08:06 Downloading - Downloading input data
2021-10-21 21:08:06 Training - Training image download completed. Training in progress.
2021-10-21 21:08:06 Uploading - Uploading generated training model
2021-10-21 21:08:06 Completed - Training job completed

2021-10-21 17:32:49 Starting - Preparing the instances for training
2021-10-21 17:32:49 Downloading - Downloading input data
2021-10-21 17:32:49 Training - Training image download completed. Training in progress.
2021-10-21 17:32:49 

### Deploying Best Pipeline

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. You can customize the initial instance count and instance type used to deploy this model.
2. Endpoint name can be changed to avoid conflict with existing endpoints.

</div>

Finally, deploy the model to SageMaker to make it functional.

In [46]:
%%time
pipeline_model.deploy(initial_instance_count=1,
                      instance_type='ml.m5.2xlarge',
                      endpoint_name=pipeline_model.name,
                      wait=True)

2021-10-22 01:57:37,570 INFO sagemaker: Creating model with name: mangrove-automl-2013
2021-10-22 01:57:38,310 INFO sagemaker: Creating endpoint with name mangrove-automl-2013
---------------!CPU times: user 434 ms, sys: 47.1 ms, total: 481 ms
Wall time: 7min 32s


Congratulations! Now you could visit the sagemaker
[endpoint console page](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/endpoints) to find the deployed endpoint (it'll take a few minutes to be in service).

<div class="alert alert-warning">
    <strong>To rerun this notebook, delete or change the name of your endpoint!</strong> <br>
If you rerun this notebook, you'll run into an error on the last step because the endpoint already exists. You can either delete the endpoint from the endpoint console page or you can change the <code>endpoint_name</code> in the previous code block.
</div>